In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import sklearn as sk
from sklearn.model_selection import train_test_split
eps=np.finfo(float).eps
from binarytree import tree,Node
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from operator import itemgetter
import copy
import collections
from pylab import *
import matplotlib
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [2]:
names=['ID','Age','Experience','Income(in p.a)','Zip','FamilySize','Expenditure(p.m)','EduLevel','MortageValue','Personalloan','SecurityAcc','CD account','InternetBanking','CreditCard']
df=pd.read_csv("LoanDataset/data.csv",names=names)
# df

In [3]:
numeric=['Age','Experience','Income(in p.a)','FamilySize','Expenditure(p.m)','EduLevel','MortageValue']
categoric=['Zip','Personalloan','SecurityAcc','CD account','InternetBanking','CreditCard']

In [4]:
X =df.drop(['Personalloan'],axis=1)
y=df['Personalloan']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
df=pd.concat([X_train,y_train],axis=1)

In [5]:
#Just seperation of class
def separateByClass(df):
#     print(dataset)
    separated = {}
    for index,row in df.iterrows():
        if  row['Personalloan'] in separated:
            separated[row['Personalloan']].append(list(row))
        else:
             separated[row['Personalloan']]=[]
    return separated

In [6]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [7]:
# sqrt((summa(x-avg)^2)/n)
def stdev(numbers):
    avg = mean(numbers)
    varianceNum=0.0
    for x in numbers:
        varianceNum=varianceNum+pow(x-avg,2)
    variance=varianceNum/float(len(numbers)-1)
#     variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [8]:
def summarize(dataset):
    summaries={}
    df1=pd.DataFrame(dataset)
    names=['ID','Age','Experience','Income(in p.a)','Zip','FamilySize','Expenditure(p.m)','EduLevel','MortageValue','SecurityAcc','CD account','InternetBanking','CreditCard','Personalloan']
    namesattr=['ID','Age','Experience','Income(in p.a)','Zip','FamilySize','Expenditure(p.m)','EduLevel','MortageValue','SecurityAcc','CD account','InternetBanking','CreditCard']
    df1.columns=names
#     print(df1)
    df1=df1.drop(['Personalloan'],axis=1)
    for attribute in namesattr:
        summaries[attribute]=(mean(df1[attribute]), stdev(df1[attribute]))
#         summaries.append((mean(df1[attribute]), stdev(df1[attribute])))
    return summaries

In [9]:
#har class ke har ek attribute ke liye mean and variance
def partitionByClass(df):
#     print(dataset)
    separated = separateByClass(df)
#     print("seperated")
#     print(seperated)
    summaries = {}
    #seperated by class list
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [10]:
#calculate gaussian distribution
def Gaussian(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [11]:
#calculate gaussian distribution of each attrbute in a given row
def getClassProbabilities(summaries,row):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for attr,meanvar in classSummaries.items():
            mean,stdev=meanvar
            x=row[attr]
            probabilities[classValue] *= Gaussian(x, mean, stdev)
    return probabilities

In [12]:
#most dominating attribute probablity
def getResultClass(summaries,row):
    probabilities = getClassProbabilities(summaries,row)
    if probabilities[0]>probabilities[1]:
        return 0
    else:
        return 1

In [13]:
def predict(summaries, testSet):
    predictions = []
    for index,row in testSet.iterrows():
        result = getResultClass(summaries,row)
        #result is basically in format of 1/0
        predictions.append(result)
    return predictions

In [14]:
# print(df)

In [15]:
summaries = partitionByClass(df)
# print(summaries)
# test model
# print(X_test)
predictions = predict(summaries, X_test)
# print(predictions)
# accuracy = getAccuracy(X_test, predictions)
# print('Accuracy: {0}%').format(accuracy)
print(confusion_matrix(y_test, predictions))  
print(classification_report(y_test,predictions)) 
print("Accuracy: ",accuracy_score(y_test, predictions))

[[701 104]
 [ 23  72]]
              precision    recall  f1-score   support

           0       0.97      0.87      0.92       805
           1       0.41      0.76      0.53        95

   micro avg       0.86      0.86      0.86       900
   macro avg       0.69      0.81      0.72       900
weighted avg       0.91      0.86      0.88       900

Accuracy:  0.8588888888888889


In [16]:
#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 
# Model Accuracy, how often is the classifier correct?
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))

[[737  68]
 [ 37  58]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       805
           1       0.46      0.61      0.52        95

   micro avg       0.88      0.88      0.88       900
   macro avg       0.71      0.76      0.73       900
weighted avg       0.90      0.88      0.89       900

Accuracy:  0.8833333333333333
